In [236]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np
from math import ceil

In [237]:
from szp_funcs import load_month, re_szp, re_szp_job, load_groups, print_df, months_year, custom_create_res, re_fot, re_fot_job, load_month_sum_by_job, accum_fot_job, bad_satus, path_to_docs

In [238]:
months = months_year[:6]
path_to_dir = 'C:\\Users\\PetukhovMD\\Documents\\code\\montly reports\\counting\\Accum + q stats\\'

In [239]:
months_year_rus = ['янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']

In [240]:
res = custom_create_res(months, load_month)

In [241]:
for_fot = custom_create_res(months, load_month_sum_by_job, on=['inn', 'snils', 'type', 'job'])

In [242]:
ped, isp, aup = load_groups('input')

In [243]:
fired = pd.read_excel(path_to_docs + 'fired_' + months[-1] + '.xlsx')

In [244]:
fired

,inn,snils
0,7717666051,170-395-964 90
1,7717666051,187-123-451 72
2,7717043346,001-192-838 03
3,7717043346,003-880-948 51
4,7717043346,004-188-977 53
...,...,...
9755,9718071371,164-315-658 63
9756,9718071371,172-995-923 30
9757,9718071371,185-458-671 20
9758,9718071371,186-538-653 21


In [245]:
def give_grp_from_matcher(row, matcher, name_of_return='szp'):
    if row['inn'] in matcher:
        if row['snils'] in matcher[row['inn']]:
            return row[name_of_return]
    return np.nan

In [246]:
def give_fired(row):
    if row['inn'] in fired['inn'].to_list() and row['snils'] in fired['snils'].to_list():
        return 1
    return 0 

In [247]:
res['fired'] = res.apply(lambda row: give_fired(row), axis=1)

In [248]:
res = res[res['fired'] == 0]

In [249]:
for i in range(len(months)):
    t = i + 1
    if t % 3 != 0:
        continue
    tm = months[:t]
    res['szp_q_' + str(int(t/3))] = res.apply(lambda row: re_szp(row, tm), axis=1)
    res['szp_teach_q_' + str(int(t/3))] = res.apply(lambda row: re_szp_job(row, tm, ['Учитель']), axis=1)
    res['szp_ped_q_' + str(int(t/3))] = res.apply(lambda row: re_szp_job(row, tm, ped), axis=1)
    res['fot_q_' + str(int(t/3))] = res.apply(lambda row: re_fot(row, tm), axis=1)
    res['fot_aup_q_' + str(int(t/3))] = res.apply(lambda row: re_fot_job(row, tm, aup), axis=1)
    df_all = res[(res['szp_q_' + str(int(t/3))].isna() == False)][['inn', 'snils', 'szp_q_' + str(int(t/3))]]
    df_teach = res[(res['szp_teach_q_' + str(int(t/3))].isna() == False)][['inn', 'snils', 'szp_teach_q_' + str(int(t/3))]]
    df_ped = res[(res['szp_ped_q_' + str(int(t/3))].isna() == False)][['inn', 'snils', 'szp_ped_q_' + str(int(t/3))]]
    df_all = df_all.groupby('inn')
    df_ped = df_ped.groupby('inn')
    df_teach = df_teach.groupby('inn')
    top10_all = {}
    bot10_all = {}
    top10_teach = {}
    bot10_teach = {}
    bot50_teach = {}
    bot50_ped = {}
    for group in df_all.groups:
        cnt = df_all.get_group(group)['snils'].nunique()
        top10_all[group] = df_all.get_group(group).sort_values(by='szp_q_' + str(int(t/3)), ascending=False, axis=0).head(ceil(cnt / 10))['snils'].to_list()
        bot10_all[group] = df_all.get_group(group).sort_values(by='szp_q_' + str(int(t/3)), ascending=False, axis=0).tail(ceil(cnt / 10))['snils'].to_list()
    for group in df_teach.groups:
        cnt = df_teach.get_group(group)['snils'].nunique()
        top10_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach_q_' + str(int(t/3)), ascending=False, axis=0).head(ceil(cnt / 10))['snils'].to_list()
        bot10_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach_q_' + str(int(t/3)), ascending=False, axis=0).tail(ceil(cnt / 10))['snils'].to_list()
        bot50_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach_q_' + str(int(t/3)), ascending=False, axis=0).tail(ceil(cnt / 2))['snils'].to_list()
    for group in df_ped.groups:
        cnt = df_ped.get_group(group)['snils'].nunique()
        bot50_ped[group] = df_ped.get_group(group).sort_values(by='szp_ped_q_' + str(int(t/3)), ascending=False, axis=0).tail(ceil(cnt / 2))['snils'].to_list()
    res['top10_all_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, top10_all, 'szp_q_' + str(int(t/3))), axis=1)
    res['bot10_all_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, bot10_all, 'szp_q_' + str(int(t/3))), axis=1)
    res['top10_teach_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, top10_teach, 'szp_teach_q_' + str(int(t/3))), axis=1)
    res['bot10_teach_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, bot10_teach, 'szp_teach_q_' + str(int(t/3))), axis=1)
    res['bot50_teach_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, bot50_teach, 'szp_teach_q_' + str(int(t/3))), axis=1)
    res['bot50_ped_q_' + str(int(t/3))] = res.apply(lambda row: give_grp_from_matcher(row, bot50_ped, 'szp_ped_q_' + str(int(t/3))), axis=1)

In [250]:
res['szp'] = res.apply(lambda row: re_szp(row, months), axis=1)
res['szp_teach'] = res.apply(lambda row: re_szp_job(row, months, ['Учитель']), axis=1)
res['szp_ped'] = res.apply(lambda row: re_szp_job(row, months, ped), axis=1)
res['fot'] = res.apply(lambda row: re_fot(row, months), axis=1)
res['fot_aup'] = res.apply(lambda row: re_fot_job(row, months, aup), axis=1)

In [251]:
df_all = res[(res.szp.isna() == False)][['inn', 'snils', 'szp']]
df_teach = res[(res.szp_teach.isna() == False)][['inn', 'snils', 'szp_teach']]
df_ped = res[(res.szp_ped.isna() == False)][['inn', 'snils', 'szp_ped']]

In [252]:
df_all = df_all.groupby('inn')
df_ped = df_ped.groupby('inn')
df_teach = df_teach.groupby('inn')

In [253]:
top10_all = {}
bot10_all = {}
top10_teach = {}
bot10_teach = {}
bot50_teach = {}
bot50_ped = {}

In [254]:
for group in df_all.groups:
    cnt = df_all.get_group(group)['snils'].nunique()
    top10_all[group] = df_all.get_group(group).sort_values(by='szp', ascending=False, axis=0).head(ceil(cnt / 10))['snils'].to_list()
    bot10_all[group] = df_all.get_group(group).sort_values(by='szp', ascending=False, axis=0).tail(ceil(cnt / 10))['snils'].to_list()

In [255]:
for group in df_teach.groups:
    cnt = df_teach.get_group(group)['snils'].nunique()
    top10_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach', ascending=False, axis=0).head(ceil(cnt / 10))['snils'].to_list()
    bot10_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach', ascending=False, axis=0).tail(ceil(cnt / 10))['snils'].to_list()
    bot50_teach[group] = df_teach.get_group(group).sort_values(by='szp_teach', ascending=False, axis=0).tail(ceil(cnt / 2))['snils'].to_list()

In [256]:
for group in df_ped.groups:
    cnt = df_ped.get_group(group)['snils'].nunique()
    bot50_ped[group] = df_ped.get_group(group).sort_values(by='szp_ped', ascending=False, axis=0).tail(ceil(cnt / 2))['snils'].to_list()

In [257]:
res['top10_all'] = res.apply(lambda row: give_grp_from_matcher(row, top10_all), axis=1)
res['bot10_all'] = res.apply(lambda row: give_grp_from_matcher(row, bot10_all), axis=1)
res['top10_teach'] = res.apply(lambda row: give_grp_from_matcher(row, top10_teach, 'szp_teach'), axis=1)
res['bot10_teach'] = res.apply(lambda row: give_grp_from_matcher(row, bot10_teach, 'szp_teach'), axis=1)
res['bot50_teach'] = res.apply(lambda row: give_grp_from_matcher(row, bot50_teach, 'szp_teach'), axis=1)
res['bot50_ped'] = res.apply(lambda row: give_grp_from_matcher(row, bot50_ped, 'szp_ped'), axis=1)

In [258]:
for_fot['fot'] = for_fot.apply(lambda row: re_fot(row, months), axis=1)
for_fot['fot_aup'] = for_fot.apply(lambda row: accum_fot_job(row, months, aup), axis=1)
for_fot['fot_isp'] = for_fot.apply(lambda row: accum_fot_job(row, months, isp), axis=1)

In [259]:
class_ruk = pd.read_excel(path_to_docs + months[-1] + '_class.xlsx')

In [260]:
class_ruk

,inn,snils
0,7720016173,001-173-705 86
1,7720016173,001-852-207 00
2,7720016173,001-906-648 18
3,7720016173,002-028-822 86
4,7720016173,002-028-997 07
...,...,...
37450,7722069597,165-377-477 04
37451,7722069597,167-259-918 14
37452,7722069597,167-462-257 90
37453,7722069597,172-553-817 78


In [261]:
class_ruk_inn = class_ruk['inn'].to_list()
class_ruk_snils = class_ruk['snils'].to_list()
class_ruk_check = [(i, j) for i, j in zip(class_ruk_inn, class_ruk_snils)]

In [262]:
def give_class(row):
    if (row['inn'], row['snils']) in class_ruk_check:
        return 1
    return 0 

In [263]:
res['class_ruk'] = res.apply(lambda row: give_class(row), axis=1)

In [264]:
cr_teach = res[res.class_ruk == 1][['szp_teach', 'inn']].groupby('inn').agg({
    'szp_teach': ['mean', 'min']
}).reset_index()
cr_teach.columns = ['inn', 'mean_cr', 'min_cr']
cr_teach

,inn,mean_cr,min_cr
0,5003021368,132967.872721,91390.263333
1,5003021495,137132.256511,96605.066667
2,5003021640,143558.853364,87213.840000
3,5003096290,138255.797492,97320.130000
4,5030032182,147810.264468,114738.320000
...,...,...,...
508,7743937357,133640.888636,101535.783333
509,7743939315,129557.090472,104930.220000
510,7743940590,148376.080980,106855.770000
511,9715206976,144812.911397,86385.620000


In [265]:
teach = res[['szp_teach', 'inn']].groupby('inn').agg({
    'szp_teach': ['mean', 'min']
}).reset_index()
teach.columns = ['inn', 'mean_teach', 'min_teach']
teach

,inn,mean_teach,min_teach
0,5003021368,125776.846286,70774.370000
1,5003021495,128093.454143,77157.910000
2,5003021640,137959.550920,84120.270000
3,5003096290,133322.108237,62584.356667
4,5030032182,136833.043060,84463.530000
...,...,...,...
590,7743940590,143160.343518,86891.053333
591,9705101759,NaN,NaN
592,9715206976,134672.887962,81436.823333
593,9715217689,121751.268404,85168.100000


In [266]:
res[res.snils == '063-133-632 27'][['inn','szp']]

,inn,szp


In [267]:
print_df(res[res.szp_teach > 200000][['inn', 'szp_teach']].groupby('inn').count().reset_index(), 'teach more 200 c ' + months[0] + ' по ' + months[-1])

In [268]:
print_df(res[(res.inn == 7727824902) & (res['szp_teach'].isna() == False)], 'check ' + str(7727824902))

In [269]:
df_teach_cr = res[(res.szp_teach.isna() == False) & (res['class_ruk'] == 1)][['inn', 'snils', 'szp_teach']].groupby('inn')
bot10_teach_cr = {}
for group in df_teach_cr.groups:
    cnt = df_teach_cr.get_group(group)['snils'].nunique()
    bot10_teach_cr[group] = df_teach_cr.get_group(group).sort_values(by='szp_teach', ascending=False, axis=0).tail(ceil(cnt / 10))['snils'].to_list()
res['bot10_teach_cr'] = res.apply(lambda row: give_grp_from_matcher(row, bot10_teach_cr, 'szp_teach'), axis=1)

In [270]:
cnts = res.groupby('inn')[['top10_all', 'top10_teach', 'bot50_teach', 'bot50_ped']].count().reset_index().rename(columns={'top10_all': 'dezil_cnt', 'top10_teach': 'dezil_teach_cnt', 'bot50_teach': 'teach_diff_cnt', 'bot50_ped': 'ped_diff_cnt'})
means = res.groupby('inn')[['szp', 'szp_ped', 'szp_teach', 'top10_all', 'bot10_all', 'top10_teach', 'bot10_teach', 'bot50_teach', 'bot50_ped', 'bot10_teach_cr']].mean().reset_index()
stats = pd.merge(means, cnts)
stats['dezil'] = stats.top10_all / stats.bot10_all
stats['dezil_teach'] = stats.top10_teach / stats.bot10_teach
stats['teach_diff'] = stats.szp_teach / stats.bot50_teach
print_df(pd.merge(stats, for_fot[for_fot.type != 'Внешнее совместительство'].groupby('inn')[['fot', 'fot_aup', 'fot_isp']].sum().reset_index()), path_to_dir + 'stats c ' +  months[0] + ' по ' + months[-1])

In [271]:
print_df(pd.merge(pd.merge(stats, teach, how='left'), cr_teach, how='left'), path_to_dir + 'stats for 7 vals с ' + months[0] + ' по ' + months[-1])

In [272]:
static_names = {'inn': (np.nan, 'ИНН'), 'snils': (np.nan, 'СНИЛС'), 'szp': (np.nan, 'СЗП'), 'szp_teach': (np.nan, 'СЗП учителей'),
'szp_ped': (np.nan, 'СЗП педы'), 'fot': (np.nan, 'ФОТ'), 'fot_aup': (np.nan, 'ФОТ АУП'), 'top10_all': (np.nan, 'max 10% от всех работников'),
'bot10_all': (np.nan, 'min 10% от всех работников'), 'top10_teach': (np.nan, 'max 10% учителей'), 'bot10_teach': (np.nan, 'min 10% учителей'),
'bot50_teach': (np.nan, 'min 50% учителей'), 'bot50_ped': (np.nan, 'min 50% педагогов'), 'fot_isp': (np.nan, 'ФОТ НОУВП')}
per_month_names = {'sum': (np.nan, 'Начисления'), 'job': (np.nan, 'Должность'), 'day': (np.nan, 'Отработанный месяц'),
'type': (np.nan, 'Вид занятости'), 'stv': (np.nan, 'Ставка'), 'status': (np.nan, 'Статус на конец месяца'), 'status_pref': (np.nan, 'Статус на начало месяца'),
'wday': (np.nan, 'Отработано дней'), 'nday': (np.nan, 'Всего рабочих дней в месяце'), 'sum_all': (np.nan, 'Начисления всего'), 'spec': (np.nan, 'Направление деятельности')}
monthly_names = {key: value for key, value in zip([key + '_' + month for key in per_month_names.keys() for month in months_year], [(month, value[1]) for value in per_month_names.values() for month in months_year_rus])}
q_names = {key: value for key, value in zip([key + '_q_' + str(q) for key in static_names.keys() for q in range(1, 5)], [('кв_' + str(q), value[1]) for value in static_names.values() for q in range(1, 5)])}
translate = static_names | per_month_names | monthly_names | q_names

In [273]:
def print_with_trans(df_in, file, trans):
    df = df_in.copy()
    df.columns = pd.MultiIndex.from_tuples(df.rename(columns=trans).columns.to_list())
    row = [[] for i in df.columns.names]
    wb = Workbook()
    ws = wb.active
    for i in df.columns:
        for j in range(len(row)):
            row[j].append(i[j])
    for i in row:
        ws.append(i)
    for i in df_to_row(df, header=False, index=False):
        ws.append(i)
    wb.save(file + '.xlsx')

In [274]:
print_with_trans(for_fot, path_to_dir + 'accum_fot c ' +  months[0] + ' по ' + months[-1] + ' rus', translate)
print_with_trans(res, path_to_dir + 'accum c ' +  months[0] + ' по ' + months[-1] + ' rus', translate)

In [275]:
df = res[res.inn == 7701050524]
cnt = df['snils'].nunique()
df.sort_values(by='szp', ascending=False)['szp'].head(ceil(cnt / 10)).mean() / df.sort_values(by='szp', ascending=False)['szp'].tail(ceil(cnt / 10)).mean()

3.1531565026591695

In [276]:
#print_df(df, path_to_dir + 'check school info ' + str(7701050524))

In [277]:
#print_df(res, path_to_dir + 'accum по ' + months[-1])

In [278]:
#print_df(for_fot, path_to_dir + 'accum_fot по ' + months[-1])